In [129]:
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import random
import scipy.stats as st
import sklearn.datasets as dta
# Dependencies
import json
import requests


In [130]:
csv_path = "archive/imdb_top_1000.csv"

# Import the CSV into a pandas DataFrame
topmovies_df = pd.read_csv(csv_path, low_memory=False)


In [131]:
#drop null rows
#no_null_topmovies_df = topmovies_df.dropna(how='any')
#no_null_topmovies_df.count()

In [132]:
cleaned_topmovies_df=topmovies_df[["Genre", "IMDB_Rating"]]
cleaned_topmovies_df.head()

,Genre,IMDB_Rating
0,Drama,9.3
1,"Crime, Drama",9.2
2,"Action, Crime, Drama",9.0
3,"Crime, Drama",9.0
4,"Crime, Drama",9.0


In [133]:
genre_values=topmovies_df["Genre"].value_counts()
genre_values

Drama                          85
Drama, Romance                 37
Comedy, Drama                  35
Comedy, Drama, Romance         31
Action, Crime, Drama           30
                               ..
Animation, Biography, Crime     1
Drama, History, Music           1
Drama, History, Mystery         1
Horror, Mystery, Sci-Fi         1
Animation, Drama, Romance       1
Name: Genre, Length: 202, dtype: int64

In [134]:
cleaned_topmovies_df.astype({'IMDB_Rating': 'str'}).dtypes

Genre          object
IMDB_Rating    object
dtype: object

In [135]:
result = cleaned_topmovies_df.set_index('IMDB_Rating').apply(lambda col:
    col.str.split(', ?', expand=True).stack())\
    .droplevel(1).reset_index().fillna('')

result

,IMDB_Rating,Genre
0,9.3,Drama
1,9.2,Crime
2,9.2,Drama
3,9.0,Action
4,9.0,Crime
...,...,...
2536,7.6,Drama
2537,7.6,War
2538,7.6,Crime
2539,7.6,Mystery


In [136]:
genre_values=result["Genre"].value_counts()
genre_values

Drama        724
Comedy       233
Crime        209
Adventure    196
Action       189
Thriller     137
Romance      125
Biography    109
Mystery       99
Animation     82
Sci-Fi        67
Fantasy       66
Family        56
History       56
War           51
Music         35
Horror        32
Western       20
Sport         19
Film-Noir     19
Musical       17
Name: Genre, dtype: int64

In [137]:
#Count of Genre Types in alphabetical order (series)
genre_values_sorted = genre_values.sort_index(axis=0, ascending=True)
genre_values_sorted

Action       189
Adventure    196
Animation     82
Biography    109
Comedy       233
Crime        209
Drama        724
Family        56
Fantasy       66
Film-Noir     19
History       56
Horror        32
Music         35
Musical       17
Mystery       99
Romance      125
Sci-Fi        67
Sport         19
Thriller     137
War           51
Western       20
Name: Genre, dtype: int64

In [138]:
genre_values_sorted.count()

21

In [128]:
# list format of Count of Genre Types in alphabetical order
#genre_count=genre_values_sorted.tolist()
#genre_count
    

In [139]:
#Sum of the ratings in alphabetical order
grouped_genre_values = result.groupby("Genre")["IMDB_Rating"].sum()
grouped_genre_values


Genre
Action       1502.3
Adventure    1558.6
Animation     650.3
Biography     865.0
Comedy       1841.5
Crime        1662.5
Drama        5762.6
Family        443.1
Fantasy       523.5
Film-Noir     151.8
History       445.4
Horror        252.4
Music         277.0
Musical       135.1
Mystery       788.8
Romance       990.7
Sci-Fi        534.5
Sport         150.6
Thriller     1083.6
War           408.7
Western       160.0
Name: IMDB_Rating, dtype: float64

In [140]:
grouped_genre_values.count()

21

In [152]:
#Sum of the ratings in alphabetial order into a dataframe, and column is renamed. 
grouped_genre_values_df=grouped_genre_values.to_frame()
grouped_genre_values_df.rename(columns={"IMDB_Rating": "Sum_Rating_By_Genre_Type"})
grouped_genre_values_df

,IMDB_Rating
Genre,
Action,1502.3
Adventure,1558.6
Animation,650.3
Biography,865.0
Comedy,1841.5
Crime,1662.5
Drama,5762.6
Family,443.1
Fantasy,523.5


In [142]:
type(grouped_genre_values_df)

pandas.core.frame.DataFrame

In [147]:
counted_genre_types_df=genre_values_sorted.to_frame()
counted_genre_types_df

,Genre
Action,189
Adventure,196
Animation,82
Biography,109
Comedy,233
Crime,209
Drama,724
Family,56
Fantasy,66
Film-Noir,19


In [148]:
genre_values_sorted_df.count()

Genre    21
dtype: int64

In [149]:
type(counted_genre_types_df)

pandas.core.frame.DataFrame

In [156]:
joined_result = pd.concat([grouped_genre_values_df, counted_genre_types_df], axis=1, join='inner')
display(joined_result)

,IMDB_Rating,Genre
Action,1502.3,189
Adventure,1558.6,196
Animation,650.3,82
Biography,865.0,109
Comedy,1841.5,233
Crime,1662.5,209
Drama,5762.6,724
Family,443.1,56
Fantasy,523.5,66
Film-Noir,151.8,19


In [158]:
joined_result=joined_result.rename(columns={"Genre": "Genre_Count"})

In [160]:
joined_result = joined_result.rename_axis('Genre')


In [161]:
joined_result

,IMDB_Rating,Genre_Count
Genre,,
Action,1502.3,189
Adventure,1558.6,196
Animation,650.3,82
Biography,865.0,109
Comedy,1841.5,233
Crime,1662.5,209
Drama,5762.6,724
Family,443.1,56
Fantasy,523.5,66


In [162]:
total_IMDB_Rating_for_all_genre_types = joined_result["IMDB_Rating"].sum()
total_IMDB_Rating_for_all_genre_types

20188.00000000002

In [163]:
total_Genre_Count_for_all_genre_types = joined_result["Genre_Count"].sum()
total_Genre_Count_for_all_genre_types 

2541

In [170]:
Averages=joined_result["IMDB_Rating"]/joined_result["Genre_Count"]



In [171]:

joined_result["Averages_By_Genre"] = Averages

In [172]:
joined_result

,IMDB_Rating,Genre_Count,Averages_By_Genre
Genre,,,
Action,1502.3,189,7.948677
Adventure,1558.6,196,7.952041
Animation,650.3,82,7.930488
Biography,865.0,109,7.935780
Comedy,1841.5,233,7.903433
Crime,1662.5,209,7.954545
Drama,5762.6,724,7.959392
Family,443.1,56,7.912500
Fantasy,523.5,66,7.931818


In [173]:
joined_result.describe()

,IMDB_Rating,Genre_Count,Averages_By_Genre
count,21.000000,21.000000,21.000000
mean,961.333333,121.000000,7.944809
std,1223.556956,153.785565,0.032582
min,135.100000,17.000000,7.887500
25%,277.000000,35.000000,7.925600
50%,534.500000,67.000000,7.947059
75%,1083.600000,137.000000,7.959392
max,5762.600000,724.000000,8.013725


In [175]:
joined_result.info()

<class 'pandas.core.frame.DataFrame'>
Index: 21 entries, Action to Western
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   IMDB_Rating        21 non-null     float64
 1   Genre_Count        21 non-null     int64  
 2   Averages_By_Genre  21 non-null     float64
dtypes: float64(2), int64(1)
memory usage: 1.2+ KB


In [180]:
joined_result['Averages_By_Genre'].max()


8.013725490196078

In [183]:
#Genre with the highest IMDB_Rating Average
highest_genre=joined_result[ joined_result['Averages_By_Genre'] == joined_result['Averages_By_Genre'].max() ]
highest_genre

,IMDB_Rating,Genre_Count,Averages_By_Genre
Genre,,,
War,408.7,51,8.013725


In [185]:
#Genre with the highest IMDB_Rating
highest_rating=joined_result[ joined_result['IMDB_Rating'] == joined_result['IMDB_Rating'].max() ]
highest_rating


,IMDB_Rating,Genre_Count,Averages_By_Genre
Genre,,,
Drama,5762.6,724,7.959392


In [187]:
Higthest_Genre_Count=joined_result[ joined_result['Genre_Count'] == joined_result['Genre_Count'].max() ]
Higthest_Genre_Count

,IMDB_Rating,Genre_Count,Averages_By_Genre
Genre,,,
Drama,5762.6,724,7.959392


In [188]:
topmovies_df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000"
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN


In [198]:
gross_cleaned_topmovies_df=topmovies_df[["Genre", "Gross"]]
gross_cleaned_topmovies_df.head()

,Genre,Gross
0,Drama,"28,341,469"
1,"Crime, Drama","134,966,411"
2,"Action, Crime, Drama","534,858,444"
3,"Crime, Drama","57,300,000"
4,"Crime, Drama","4,360,000"


In [206]:
result_gross = gross_cleaned_topmovies_df.set_index('Gross').apply(lambda col:
    col.str.split(', ?', expand=True).stack())\
    .droplevel(1).reset_index().fillna('')

result_gross

,Gross,Genre
0,"28,341,469",Drama
1,"134,966,411",Crime
2,"134,966,411",Drama
3,"534,858,444",Action
4,"534,858,444",Crime
...,...,...
2536,,Drama
2537,,War
2538,,Crime
2539,,Mystery


In [207]:
#Count of Genre Types & Gross
genre_gross_values=result_gross["Genre"].value_counts()
genre_gross_values

Drama        724
Comedy       233
Crime        209
Adventure    196
Action       189
Thriller     137
Romance      125
Biography    109
Mystery       99
Animation     82
Sci-Fi        67
Fantasy       66
Family        56
History       56
War           51
Music         35
Horror        32
Western       20
Sport         19
Film-Noir     19
Musical       17
Name: Genre, dtype: int64

In [208]:
#Count of Genre Types & Gross in alphabetical order (series)
genre_gross_values_sorted = genre_values.sort_index(axis=0, ascending=True)
genre_gross_values_sorted

Action       189
Adventure    196
Animation     82
Biography    109
Comedy       233
Crime        209
Drama        724
Family        56
Fantasy       66
Film-Noir     19
History       56
Horror        32
Music         35
Musical       17
Mystery       99
Romance      125
Sci-Fi        67
Sport         19
Thriller     137
War           51
Western       20
Name: Genre, dtype: int64

In [235]:
#Sum of the gross in alphabetical order
grouped_genre_gross_values = result_gross.groupby("Genre")["Gross"].sum()
grouped_genre_gross_values


Genre
Action       534,858,444377,845,905292,576,195315,544,75034...
Adventure    377,845,905292,576,195315,544,750342,551,36529...
Animation    10,055,859422,783,7775,017,246190,241,310209,7...
Biography    96,898,81846,836,39413,182,28132,572,57712,391...
Comedy       53,367,84457,598,24713,182,281210,609,762163,2...
Crime        134,966,411534,858,44457,300,0004,360,000107,9...
Drama        28,341,469134,966,411534,858,44457,300,0004,36...
Family       10,055,859163,245209,726,0151,223,869223,808,1...
Fantasy      290,475,067136,801,374322,740,1405,017,246309,...
Film-Noir    5,720,0003,969,893449,1912,237,659654,0004,360...
History      96,898,81875,600,00051,973,0295,509,04012,100,...
Horror       32,000,00044,017,37478,900,00013,782,8384,050,...
Music        13,092,00032,572,5771,373,94325,000,0001,498,2...
Musical      8,819,02810,616,10470,1471,787,3784,184,03680,...
Mystery      100,125,64353,089,89123,341,56832,000,00011,28...
Romance      330,252,18257,598,24711,990,4011,024

In [216]:
#Sum of the gross in alphabetial order into a dataframe, and column is renamed. 
grouped_genre_gross_values_df=grouped_genre_gross_values.to_frame()
grouped_genre_gross_values_df

,Gross
Genre,
Action,"534,858,444377,845,905292,576,195315,544,75034..."
Adventure,"377,845,905292,576,195315,544,750342,551,36529..."
Animation,"10,055,859422,783,7775,017,246190,241,310209,7..."
Biography,"96,898,81846,836,39413,182,28132,572,57712,391..."
Comedy,"53,367,84457,598,24713,182,281210,609,762163,2..."
Crime,"134,966,411534,858,44457,300,0004,360,000107,9..."
Drama,"28,341,469134,966,411534,858,44457,300,0004,36..."
Family,"10,055,859163,245209,726,0151,223,869223,808,1..."
Fantasy,"290,475,067136,801,374322,740,1405,017,246309,..."


In [217]:
#Sum of the genre by Genre Type
counted_genre_gross_values_df=genre_gross_values_sorted.to_frame()
counted_genre_gross_values_df

,Genre
Action,189
Adventure,196
Animation,82
Biography,109
Comedy,233
Crime,209
Drama,724
Family,56
Fantasy,66
Film-Noir,19


In [219]:
#Concat the two dataframes together to get the gross revenue by the each genre type
joined_gross_result = pd.concat([grouped_genre_gross_values_df,counted_genre_gross_values_df], axis=1, join='inner')
display(joined_gross_result)

,Gross,Genre
Action,"534,858,444377,845,905292,576,195315,544,75034...",189
Adventure,"377,845,905292,576,195315,544,750342,551,36529...",196
Animation,"10,055,859422,783,7775,017,246190,241,310209,7...",82
Biography,"96,898,81846,836,39413,182,28132,572,57712,391...",109
Comedy,"53,367,84457,598,24713,182,281210,609,762163,2...",233
Crime,"134,966,411534,858,44457,300,0004,360,000107,9...",209
Drama,"28,341,469134,966,411534,858,44457,300,0004,36...",724
Family,"10,055,859163,245209,726,0151,223,869223,808,1...",56
Fantasy,"290,475,067136,801,374322,740,1405,017,246309,...",66
Film-Noir,"5,720,0003,969,893449,1912,237,659654,0004,360...",19


In [224]:
joined_gross_result=joined_gross_result.rename(columns={"Genre": "Genre_Count"})
joined_gross_result=joined_gross_result.rename(columns={"Gross": "Total_Gross"})
joined_gross_result = joined_gross_result.rename_axis('Genre')


In [225]:
joined_gross_result

,Total_Gross,Genre_Count
Genre,,
Action,"534,858,444377,845,905292,576,195315,544,75034...",189
Adventure,"377,845,905292,576,195315,544,750342,551,36529...",196
Animation,"10,055,859422,783,7775,017,246190,241,310209,7...",82
Biography,"96,898,81846,836,39413,182,28132,572,57712,391...",109
Comedy,"53,367,84457,598,24713,182,281210,609,762163,2...",233
Crime,"134,966,411534,858,44457,300,0004,360,000107,9...",209
Drama,"28,341,469134,966,411534,858,44457,300,0004,36...",724
Family,"10,055,859163,245209,726,0151,223,869223,808,1...",56
Fantasy,"290,475,067136,801,374322,740,1405,017,246309,...",66


In [226]:
Averages=joined_gross_result["Total_Gross"]/joined_gross_result["Genre_Count"]

TypeError: unsupported operand type(s) for /: 'str' and 'int'

In [229]:
joined_gross_result.dtypes

Total_Gross    object
Genre_Count     int64
dtype: object

In [237]:
joined_gross_result.astype({'Total_Gross': 'float'}).dtypes

ValueError: could not convert string to float: '534,858,444377,845,905292,576,195315,544,750342,551,365171,479,930290,475,067322,740,140269,061187,705,42719,501,238204,843,35012,391,761190,241,310858,373,000678,815,482448,139,099707,4812,375,308248,159,97185,160,248309,125,4094,186,16820,186,65939,56713,657,11570,511,035206,852,4323,897,569197,171,80683,008,8524,135,750977,375901,610226,277,06826,947,624117,624,028154,058,340217,581,2311,111,06170,099,045402,453,88232,868,9431,033,8958,178,0016,738,0002,625,803333,176,60092,054,159183,637,894100,119363,070,7098,060623,279,547227,471,070167,445,96066,208,183169,659305,413,918261,441,09223,159,3053,296515,905553,17138,400,00014,500,00012,535,000315,058,289936,662,225233,921,534100,206,256115,646,235257,730,01913,756,082318,412,10153,710,0191,000,045121,661,68357,229,8905,100,00012,064,4723,981,000188,373,16148,023,088532,177,324408,084,349222,527,828177,002,924128,3926,460145,000,989760,507,62550,866,63523,637,26525,514,51754,234,062128,

In [238]:
joined_gross_result['Total_Gross'] = joined_gross_result['Total_Gross'].astype(float, errors = 'raise')

ValueError: could not convert string to float: '534,858,444377,845,905292,576,195315,544,750342,551,365171,479,930290,475,067322,740,140269,061187,705,42719,501,238204,843,35012,391,761190,241,310858,373,000678,815,482448,139,099707,4812,375,308248,159,97185,160,248309,125,4094,186,16820,186,65939,56713,657,11570,511,035206,852,4323,897,569197,171,80683,008,8524,135,750977,375901,610226,277,06826,947,624117,624,028154,058,340217,581,2311,111,06170,099,045402,453,88232,868,9431,033,8958,178,0016,738,0002,625,803333,176,60092,054,159183,637,894100,119363,070,7098,060623,279,547227,471,070167,445,96066,208,183169,659305,413,918261,441,09223,159,3053,296515,905553,17138,400,00014,500,00012,535,000315,058,289936,662,225233,921,534100,206,256115,646,235257,730,01913,756,082318,412,10153,710,0191,000,045121,661,68357,229,8905,100,00012,064,4723,981,000188,373,16148,023,088532,177,324408,084,349222,527,828177,002,924128,3926,460145,000,989760,507,62550,866,63523,637,26525,514,51754,234,062128,